In [6]:
# %pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Brian\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
import pandas as pd
import yaml
import os

WIDTH = 640 #will replace w actual
HEIGHT = 480 #will also replace w actual

#load in clean dataset
df = pd.read_csv(r"C:\Users\Brian\.vscode\DSProject3\SCRIPTS\cleaned_foid.csv")

class_map = {
    'Albacore':0, 'Bigeye tuna':1, 'Black marlin':2, 'Blue marlin':3, 'Great barracuda':4, 'Human':5,
    'Indo Pacific sailfish':6, 'Long snouted lancetfish':7, 'Mahi mahi':8, 'Mola mola':9, 'No fish':10,
    'Oilfish':11, 'Opah':12, 'Pelagic stingray':13, 'Pomfret':14, 'Rainbow runner':15, 'Roudie scolar':16, 'Shark':17,
    'Shortbill spearfish':18, 'Sickle pomfret':19, 'Skipjack tuna':20, 'Snake mackerel':21, 'Striped marlin':22,
    'Swordfish':23, 'Thresher shark':24, 'Unknown':25, 'Wahoo':26, 'Yellowfin tuna':27
}

df["class_id"] = df["label"].map(class_map)

# Normalize coordinates
df["x_center_norm"] = df["center_x"] / WIDTH
df["y_center_norm"] = df["center_y"] / HEIGHT
df["width_norm"] = df["width"] / WIDTH
df["height_norm"] = df["height"] / HEIGHT

for img_id, group in df.groupby("img_id"):
    label_path = f"C:/Users/Brian/.vscode/DSProject3/SCRIPTS/labels/{img_id}.txt"
    with open(label_path, "w") as f:
        for _, row in group.iterrows():
            f.write(f"{row['class_id']} {row['x_center_norm']:.6f} {row['y_center_norm']:.6f} {row['width_norm']:.6f} {row['height_norm']:.6f}\n")
    

In [23]:
# Define paths 
train_path = 'C:/Users/Brian/.vscode/DSProject3/images/train'
val_path = 'C:/Users/Brian/.vscode/DSProject3/images/val'
test_path = 'C:/Users/Brian/.vscode/DSProject3/images/test'

# Get list of class names from the class_map (in correct order)
class_names = [None] * len(class_map)
for name, idx in class_map.items():
    class_names[idx] = name

# Create dictionary for YAML
data_yaml = {
    'val': val_path.replace('\\', '/'),
    'train': train_path.replace('\\', '/'),
    'test': test_path.replace('\\', '/'),
    'names': class_names,
    'nc': len(class_map)
}

# Write to YAML
with open('C:/Users/Brian/.vscode/DSProject3/SCRIPTS/data.yaml', 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

In [2]:
from ultralytics import YOLO

#pretrained model
model = YOLO("yolov8s.pt")

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Brian\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████████████████████████████████████████████████████████████████████████████████| 21.5M/21.5M [00:00<00:00, 32.4MB/s]


In [3]:
model.info()

YOLOv8s summary: 129 layers, 11,166,560 parameters, 0 gradients, 28.8 GFLOPs


(129, 11166560, 0, 28.816844800000002)

In [24]:
model.train(
    data=r"C:\Users\Brian\.vscode\DSProject3\SCRIPTS\data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    name="fishnet_yolov8",
    device=0
    workers=4    
)

Ultralytics 8.3.109  Python-3.11.9 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-1145G7 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\Brian\.vscode\DSProject3\SCRIPTS\data.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=fishnet_yolov83, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=

RuntimeError: Dataset 'C://Users/Brian/.vscode/DSProject3/SCRIPTS/data.yaml' error  
Dataset 'C://Users/Brian/.vscode/DSProject3/SCRIPTS/data.yaml' images not found , missing path 'C:\Users\Brian\.vscode\DSProject3\images\val'
Note dataset download directory is 'C:\Users\Brian\.vscode\DSProject3\SCRIPTS\datasets'. You can update this in 'C:\Users\Brian\AppData\Roaming\Ultralytics\settings.json'

In [ ]:
metrics = model.val()